In [1]:
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import json
from io import StringIO
import pygwalker as pyg
import os
from tqdm import tqdm
import polars.selectors as cs
from scipy.stats import zscore

reading competition editions json

In [2]:
ce_json_path = './skillcorner-competition-editions.json'
ce_df1 = pl.read_json(ce_json_path)
ce_df1.head(1)
#ce_df.columns

count,next,previous,results
i64,null,null,list[struct[4]]
938,null,null,"[{0,{0,""FIFA"",""World Cup"",""male"",""adult""},{10,2014,2014,""2014""},""FIFA - World Cup - 2014""}, {1,{1,""ENG"",""Premier League"",""male"",""adult""},{2,2015,2016,""2015/2016""},""ENG - Premier League - 2015/2016""}, … {1129,{221,""INT"",""Pinatar National Team Friendlies"",""male"",""U21""},{128,2025,2025,""2025""},""INT - Pinatar National Team Friendlies - 2025""}]"


In [3]:
#ce_df2 = ce_df1.explode('results')
ce_df2 = ce_df1.select('results').explode('results')
ce_df2 = ce_df2.unnest('results')
ce_df2.head(1)

id,competition,season,name
i64,struct[5],struct[4],str
0,"{0,""FIFA"",""World Cup"",""male"",""adult""}","{10,2014,2014,""2014""}","""FIFA - World Cup - 2014"""


In [4]:
ce_name_id = ce_df2.select([
    pl.col('name'),
    pl.col('id'),
    ]
)
ce_name_id.head(1)

name,id
str,i64
"""FIFA - World Cup - 2014""",0


In [5]:
ce_competition = ce_df2.select('competition').unnest('competition')
ce_competition = ce_competition.rename({col: f"competition_{col}" for col in ce_competition.columns})
ce_competition.head(1)

competition_id,competition_area,competition_name,competition_gender,competition_age_group
i64,str,str,str,str
0,"""FIFA""","""World Cup""","""male""","""adult"""


In [6]:
ce_season = ce_df2.select('season').unnest('season')
ce_season = ce_season.rename({col: f"season_{col}" for col in ce_season.columns})
ce_season.head(1)

season_id,season_start_year,season_end_year,season_name
i64,i64,i64,str
10,2014,2014,"""2014"""


In [7]:
ce = pl.concat([
    ce_name_id,
    ce_season,
    ce_competition
],
how = "horizontal",)
ce.head(10)

name,id,season_id,season_start_year,season_end_year,season_name,competition_id,competition_area,competition_name,competition_gender,competition_age_group
str,i64,i64,i64,i64,str,i64,str,str,str,str
"""FIFA - World Cup - 2014""",0,10,2014,2014,"""2014""",0,"""FIFA""","""World Cup""","""male""","""adult"""
"""ENG - Premier League - 2015/20…",1,2,2015,2016,"""2015/2016""",1,"""ENG""","""Premier League""","""male""","""adult"""
"""UEFA - European Championship -…",2,12,2016,2016,"""2016""",2,"""UEFA""","""European Championship""","""male""","""adult"""
"""ENG - Premier League - 2016/20…",3,3,2016,2017,"""2016/2017""",1,"""ENG""","""Premier League""","""male""","""adult"""
"""ENG - Premier League - 2017/20…",4,4,2017,2018,"""2017/2018""",1,"""ENG""","""Premier League""","""male""","""adult"""
"""FRA - Ligue 1 - 2016/2017""",5,3,2016,2017,"""2016/2017""",3,"""FRA""","""Ligue 1""","""male""","""adult"""
"""FRA - Ligue 1 - 2017/2018""",6,4,2017,2018,"""2017/2018""",3,"""FRA""","""Ligue 1""","""male""","""adult"""
"""ESP - LaLiga - 2016/2017""",7,3,2016,2017,"""2016/2017""",4,"""ESP""","""LaLiga""","""male""","""adult"""
"""ESP - LaLiga - 2017/2018""",8,4,2017,2018,"""2017/2018""",4,"""ESP""","""LaLiga""","""male""","""adult"""


In [8]:
# checking if columns contain more than 1 value
#ce.select("competition_age_group").unique()

reading json player file- test on 1 file


In [9]:
position_json_testpath = './skillcorner-20250214/metrics-ce249-rAM.json'

position_test_df = pl.read_json(position_json_testpath)

In [10]:
position_test_df.columns
#position_test_df.head(10)

['Count Medium Acceleration OTIP',
 'Count Medium Acceleration TIP',
 'Count High Deceleration TIP 2',
 'team_name',
 'Running Distance TIP',
 'HSR Distance 2',
 'Count Sprint 2',
 'Minutes OTIP',
 'player_birthdate',
 'Count Sprint',
 'Count HSR TIP',
 'competition_edition_id',
 'Running Distance',
 'Sprinting Distance TIP 2',
 'Count Sprint TIP',
 'Sprinting Distance OTIP 2',
 'Count Medium Deceleration OTIP 1',
 'Count High Acceleration TIP',
 'Count Medium Acceleration OTIP 1',
 'Count High Acceleration OTIP',
 'Running Distance OTIP',
 'Minutes OTIP 1',
 'Minutes OTIP 2',
 'position',
 'Count High Deceleration OTIP 1',
 'Count High Acceleration OTIP 1',
 'Minutes TIP 2',
 'Count High Acceleration 2',
 'Count High Deceleration 2',
 'Distance OTIP 2',
 'Sprinting Distance',
 'Running Distance 2',
 'Distance 1',
 'date',
 'Count Sprint TIP 2',
 'team_id',
 'Count HSR 1',
 'Sprinting Distance TIP',
 'Count HSR 2',
 'Count High Deceleration TIP 1',
 'Count Medium Deceleration OTIP',
 '

In [11]:
#define the path to the entire directory
json_dir_path = './skillcorner-20250214/'

### Look at correlations between parameters
Looking at the correlations across all competitions across all positions to see if we can leave out certain parameters

In [12]:
json_list = os.listdir(json_dir_path)
#json_list

In [13]:
read_json_list = []

for jsonf in tqdm(json_list):
    json_path = f"{json_dir_path}{jsonf}"
    json_df = pl.read_json(json_path)

    read_json_list.append(json_df)

corr_df = pl.concat(read_json_list)

  0%|          | 4/1652 [00:00<00:46, 35.56it/s]

100%|██████████| 1652/1652 [02:13<00:00, 12.42it/s]


In [14]:
corr_df.shape

(449537, 117)

In [15]:
corr_df.head()

Count Medium Acceleration OTIP,Count Medium Acceleration TIP,Count High Deceleration TIP 2,team_name,Running Distance TIP,HSR Distance 2,Count Sprint 2,Minutes OTIP,player_birthdate,Count Sprint,Count HSR TIP,competition_edition_id,Running Distance,Sprinting Distance TIP 2,Count Sprint TIP,Sprinting Distance OTIP 2,Count Medium Deceleration OTIP 1,Count High Acceleration TIP,Count Medium Acceleration OTIP 1,Count High Acceleration OTIP,Running Distance OTIP,Minutes OTIP 1,Minutes OTIP 2,position,Count High Deceleration OTIP 1,Count High Acceleration OTIP 1,Minutes TIP 2,Count High Acceleration 2,Count High Deceleration 2,Distance OTIP 2,Sprinting Distance,Running Distance 2,Distance 1,date,Count Sprint TIP 2,team_id,Count HSR 1,…,Sprinting Distance 1,Count High Acceleration TIP 1,group,HSR Distance,Count HSR,HSR Distance OTIP 2,Count HSR OTIP 1,competition_id,Count Medium Deceleration TIP 2,PSV-99,player_name,Sprinting Distance OTIP,team,Count High Deceleration OTIP,Count High Acceleration,Count Medium Acceleration 2,Count High Deceleration 1,Sprinting Distance TIP 1,Count HSR OTIP,Count Medium Deceleration 2,Count Medium Acceleration TIP 2,Sprinting Distance 2,Count High Deceleration TIP,Running Distance OTIP 1,Minutes TIP 1,Distance TIP 1,Count Sprint OTIP,Count HSR OTIP 2,Count High Deceleration OTIP 2,Count Sprint OTIP 1,Minutes TIP,Count Sprint OTIP 2,Count High Acceleration 1,match_name,Running Distance 1,quality_check,Count High Acceleration TIP 2
i64,i64,i64,str,i64,i64,i64,f64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,str,i64,i64,f64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,…,i64,i64,str,i64,i64,i64,i64,i64,i64,f64,str,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,f64,i64,i64,str,i64,bool,i64
47,22,1,"""Royal Antwerp FC""",661,358,2,35.57,"""1995-02-17""",5,19,249,2415,19,2,18,17,0,24,0,1504,18.27,17.3,"""AM""",3,0,11.08,1,3,2744,148,1061,6810,"""2021-07-25""",1,462,33,…,92,0,"""Midfield""",745,59,218,17,16,9,25.7,"""Pieter Gerkens""",45,"""Royal Antwerp FC""",4,4,48,8,11,34,47,10,56,3,842,13.13,1874,2,17,1,1,24.21,1,3,"""KV Mechelen v Royal Antwerp FC""",1354,true,0
32,25,1,"""Oud-Heverlee Leuven""",610,275,5,29.59,"""1989-07-25""",8,20,249,1101,119,4,0,11,2,15,1,402,14.52,15.07,"""AM""",1,0,12.32,2,3,1739,202,612,4842,"""2021-07-24""",3,27,20,…,48,1,"""Midfield""",476,39,63,10,16,11,28.3,"""Xavier Mercier""",24,"""Oud-Heverlee Leuven""",1,5,41,5,17,17,46,14,154,2,178,13.2,1660,1,7,0,1,25.52,0,3,"""Oud-Heverlee Leuven v SV Zulte…",489,true,1
23,26,1,"""SV Zulte-Waregem""",724,129,1,19.77,"""1996-01-18""",5,18,249,1241,0,2,7,13,1,17,2,425,13.2,6.57,"""AM""",2,1,7.93,2,4,938,105,527,5620,"""2021-07-24""",0,8,21,…,95,0,"""Midfield""",421,34,78,7,16,11,26.4,"""Omar Nicolás Govea García""",36,"""SV Zulte-Waregem""",3,3,25,7,26,14,25,8,10,2,253,14.52,2289,2,7,1,1,22.45,1,1,"""Oud-Heverlee Leuven v SV Zulte…",714,true,1
26,21,1,"""Oud-Heverlee Leuven""",603,310,6,23.0,"""1996-07-18""",12,25,249,1234,88,8,20,13,1,17,0,536,14.52,8.48,"""AM""",2,0,7.15,1,6,1326,207,554,5410,"""2021-07-24""",4,27,25,…,76,0,"""Midfield""",594,50,110,11,16,5,27.4,"""Siebe Schrijvers""",27,"""Oud-Heverlee Leuven""",5,1,27,9,44,20,26,8,131,3,301,13.2,1883,2,9,3,1,20.35,1,0,"""Oud-Heverlee Leuven v SV Zulte…",680,true,1
47,29,0,"""KAS Eupen""",603,81,1,37.23,"""1992-02-09""",5,17,249,1405,0,2,4,18,1,21,1,788,18.05,19.18,"""AM""",2,1,12.98,0,2,2252,60,462,6257,"""2021-07-25""",0,26,23,…,56,1,"""Midfield""",334,35,62,9,16,10,25.5,"""Stef Peeters""",24,"""KAS Eupen""",3,2,39,6,36,17,41,11,4,1,468,15.68,2340,2,8,1,1,28.66,1,2,"""Club Brugge KV v KAS Eupen""",943,true,0


In [16]:
pl.Config.set_tbl_cols(150)
corr_df.describe()

statistic,Count Medium Acceleration OTIP,Count Medium Acceleration TIP,Count High Deceleration TIP 2,team_name,Running Distance TIP,HSR Distance 2,Count Sprint 2,Minutes OTIP,player_birthdate,Count Sprint,Count HSR TIP,competition_edition_id,Running Distance,Sprinting Distance TIP 2,Count Sprint TIP,Sprinting Distance OTIP 2,Count Medium Deceleration OTIP 1,Count High Acceleration TIP,Count Medium Acceleration OTIP 1,Count High Acceleration OTIP,Running Distance OTIP,Minutes OTIP 1,Minutes OTIP 2,position,Count High Deceleration OTIP 1,Count High Acceleration OTIP 1,Minutes TIP 2,Count High Acceleration 2,Count High Deceleration 2,Distance OTIP 2,Sprinting Distance,Running Distance 2,Distance 1,date,Count Sprint TIP 2,team_id,Count HSR 1,Sprinting Distance TIP,Count HSR 2,Count High Deceleration TIP 1,Count Medium Deceleration OTIP,Distance 2,Running Distance TIP 2,Minutes,player_id,Count Medium Acceleration,Sprinting Distance OTIP 1,Count Medium Deceleration TIP 1,Count High Acceleration OTIP 2,Count Medium Deceleration OTIP 2,short_name,Count HSR TIP 2,Count HSR TIP 1,Running Distance TIP 1,Count Sprint 1,Count Medium Deceleration,Distance OTIP,Distance TIP,season_name,Count High Deceleration,match_id,HSR Distance 1,HSR Distance TIP 2,HSR Distance OTIP 1,HSR Distance OTIP,Minutes 1,Count Medium Acceleration TIP 1,Count Medium Deceleration 1,Count Medium Acceleration OTIP 2,Count Medium Acceleration 1,Distance TIP 2,Count Sprint TIP 1,Running Distance OTIP 2,HSR Distance TIP 1,Count Medium Deceleration TIP,Distance OTIP 1,season_id,Distance,HSR Distance TIP,Minutes 2,Sprinting Distance 1,Count High Acceleration TIP 1,group,HSR Distance,Count HSR,HSR Distance OTIP 2,Count HSR OTIP 1,competition_id,Count Medium Deceleration TIP 2,PSV-99,player_name,Sprinting Distance OTIP,team,Count High Deceleration OTIP,Count High Acceleration,Count Medium Acceleration 2,Count High Deceleration 1,Sprinting Distance TIP 1,Count HSR OTIP,Count Medium Deceleration 2,Count Medium Acceleration TIP 2,Sprinting Distance 2,Count High Deceleration TIP,Running Distance OTIP 1,Minutes TIP 1,Distance TIP 1,Count Sprint OTIP,Count HSR OTIP 2,Count High Deceleration OTIP 2,Count Sprint OTIP 1,Minutes TIP,Count Sprint OTIP 2,Count High Acceleration 1,match_name,Running Distance 1,quality_check,Count High Acceleration TIP 2
str,f64,f64,f64,str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64
"""count""",449537.0,449537.0,448747.0,"""449537""",449537.0,449511.0,449511.0,449537.0,"""449324""",449537.0,449537.0,449537.0,449537.0,449511.0,449537.0,449511.0,449429.0,449537.0,449429.0,449537.0,449537.0,449537.0,449537.0,"""449537""",449319.0,449319.0,449537.0,449511.0,449511.0,449511.0,449537.0,449511.0,449429.0,"""449537""",449511.0,449537.0,449429.0,449537.0,449511.0,449299.0,449537.0,449511.0,449511.0,449537.0,449537.0,449537.0,449429.0,449429.0,448984.0,449511.0,"""449537""",449511.0,449429.0,449429.0,449429.0,449537.0,449537.0,449537.0,"""449537""",449537.0,449537.0,449429.0,449511.0,449429.0,449537.0,449537.0,449429.0,449429.0,449511.0,449429.0,449511.0,449429.0,449511.0,449429.0,449537.0,449429.0,449537.0,449537.0,449537.0,449537.0,449429.0,449299.0,"""449537""",449537.0,449537.0,449511.0,449429.0,449537.0,449511.0,449537.0,"""449537""",449537.0,"""449537""",449537.0,449537.0,449511.0,449429.0,449429.0,449537.0,449511.0,449511.0,449511.0,449537.0,449429.0,449537.0,449429.0,449537.0,449511.0,448984.0,449429.0,449537.0,449511.0,449429.0,"""449537""",449429.0,449537.0,448747.0
"""null_count""",0.0,0.0,790.0,"""0""",0.0,26.0,26.0,0.0,"""213""",0.0,0.0,0.0,0.0,26.0,0

In [17]:
corr_df.select("position").unique()

position
str
"""AM"""
"""LWB"""
"""RM"""
"""LW"""
"""RF"""
…
"""LF"""
"""CF"""
"""RCB"""


#### Investigating null values

In [18]:
null_count_df = corr_df.null_count()
null_count_df

Count Medium Acceleration OTIP,Count Medium Acceleration TIP,Count High Deceleration TIP 2,team_name,Running Distance TIP,HSR Distance 2,Count Sprint 2,Minutes OTIP,player_birthdate,Count Sprint,Count HSR TIP,competition_edition_id,Running Distance,Sprinting Distance TIP 2,Count Sprint TIP,Sprinting Distance OTIP 2,Count Medium Deceleration OTIP 1,Count High Acceleration TIP,Count Medium Acceleration OTIP 1,Count High Acceleration OTIP,Running Distance OTIP,Minutes OTIP 1,Minutes OTIP 2,position,Count High Deceleration OTIP 1,Count High Acceleration OTIP 1,Minutes TIP 2,Count High Acceleration 2,Count High Deceleration 2,Distance OTIP 2,Sprinting Distance,Running Distance 2,Distance 1,date,Count Sprint TIP 2,team_id,Count HSR 1,Sprinting Distance TIP,Count HSR 2,Count High Deceleration TIP 1,Count Medium Deceleration OTIP,Distance 2,Running Distance TIP 2,Minutes,player_id,Count Medium Acceleration,Sprinting Distance OTIP 1,Count Medium Deceleration TIP 1,Count High Acceleration OTIP 2,Count Medium Deceleration OTIP 2,short_name,Count HSR TIP 2,Count HSR TIP 1,Running Distance TIP 1,Count Sprint 1,Count Medium Deceleration,Distance OTIP,Distance TIP,season_name,Count High Deceleration,match_id,HSR Distance 1,HSR Distance TIP 2,HSR Distance OTIP 1,HSR Distance OTIP,Minutes 1,Count Medium Acceleration TIP 1,Count Medium Deceleration 1,Count Medium Acceleration OTIP 2,Count Medium Acceleration 1,Distance TIP 2,Count Sprint TIP 1,Running Distance OTIP 2,HSR Distance TIP 1,Count Medium Deceleration TIP,Distance OTIP 1,season_id,Distance,HSR Distance TIP,Minutes 2,Sprinting Distance 1,Count High Acceleration TIP 1,group,HSR Distance,Count HSR,HSR Distance OTIP 2,Count HSR OTIP 1,competition_id,Count Medium Deceleration TIP 2,PSV-99,player_name,Sprinting Distance OTIP,team,Count High Deceleration OTIP,Count High Acceleration,Count Medium Acceleration 2,Count High Deceleration 1,Sprinting Distance TIP 1,Count HSR OTIP,Count Medium Deceleration 2,Count Medium Acceleration TIP 2,Sprinting Distance 2,Count High Deceleration TIP,Running Distance OTIP 1,Minutes TIP 1,Distance TIP 1,Count Sprint OTIP,Count HSR OTIP 2,Count High Deceleration OTIP 2,Count Sprint OTIP 1,Minutes TIP,Count Sprint OTIP 2,Count High Acceleration 1,match_name,Running Distance 1,quality_check,Count High Acceleration TIP 2
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,790,0,0,26,26,0,213,0,0,0,0,26,0,26,108,0,108,0,0,0,0,0,218,218,0,26,26,26,0,26,108,0,26,0,108,0,26,238,0,26,26,0,0,0,108,108,553,26,0,26,108,108,108,0,0,0,0,0,0,108,26,108,0,0,108,108,26,108,26,108,26,108,0,108,0,0,0,0,108,238,0,0,0,26,108,0,26,0,0,0,0,0,0,26,108,108,0,26,26,26,0,108,0,108,0,26,553,108,0,26,108,0,108,0,790


In [19]:
corr_df_missing = (corr_df.filter(pl.any_horizontal(pl.all().is_null())))
corr_df_missing.shape

(1824, 117)

In [20]:
corr_df_missing

Count Medium Acceleration OTIP,Count Medium Acceleration TIP,Count High Deceleration TIP 2,team_name,Running Distance TIP,HSR Distance 2,Count Sprint 2,Minutes OTIP,player_birthdate,Count Sprint,Count HSR TIP,competition_edition_id,Running Distance,Sprinting Distance TIP 2,Count Sprint TIP,Sprinting Distance OTIP 2,Count Medium Deceleration OTIP 1,Count High Acceleration TIP,Count Medium Acceleration OTIP 1,Count High Acceleration OTIP,Running Distance OTIP,Minutes OTIP 1,Minutes OTIP 2,position,Count High Deceleration OTIP 1,Count High Acceleration OTIP 1,Minutes TIP 2,Count High Acceleration 2,Count High Deceleration 2,Distance OTIP 2,Sprinting Distance,Running Distance 2,Distance 1,date,Count Sprint TIP 2,team_id,Count HSR 1,Sprinting Distance TIP,Count HSR 2,Count High Deceleration TIP 1,Count Medium Deceleration OTIP,Distance 2,Running Distance TIP 2,Minutes,player_id,Count Medium Acceleration,Sprinting Distance OTIP 1,Count Medium Deceleration TIP 1,Count High Acceleration OTIP 2,Count Medium Deceleration OTIP 2,short_name,Count HSR TIP 2,Count HSR TIP 1,Running Distance TIP 1,Count Sprint 1,Count Medium Deceleration,Distance OTIP,Distance TIP,season_name,Count High Deceleration,match_id,HSR Distance 1,HSR Distance TIP 2,HSR Distance OTIP 1,HSR Distance OTIP,Minutes 1,Count Medium Acceleration TIP 1,Count Medium Deceleration 1,Count Medium Acceleration OTIP 2,Count Medium Acceleration 1,Distance TIP 2,Count Sprint TIP 1,Running Distance OTIP 2,HSR Distance TIP 1,Count Medium Deceleration TIP,Distance OTIP 1,season_id,Distance,HSR Distance TIP,Minutes 2,Sprinting Distance 1,Count High Acceleration TIP 1,group,HSR Distance,Count HSR,HSR Distance OTIP 2,Count HSR OTIP 1,competition_id,Count Medium Deceleration TIP 2,PSV-99,player_name,Sprinting Distance OTIP,team,Count High Deceleration OTIP,Count High Acceleration,Count Medium Acceleration 2,Count High Deceleration 1,Sprinting Distance TIP 1,Count HSR OTIP,Count Medium Deceleration 2,Count Medium Acceleration TIP 2,Sprinting Distance 2,Count High Deceleration TIP,Running Distance OTIP 1,Minutes TIP 1,Distance TIP 1,Count Sprint OTIP,Count HSR OTIP 2,Count High Deceleration OTIP 2,Count Sprint OTIP 1,Minutes TIP,Count Sprint OTIP 2,Count High Acceleration 1,match_name,Running Distance 1,quality_check,Count High Acceleration TIP 2
i64,i64,i64,str,i64,i64,i64,f64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,str,i64,i64,f64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,str,i64,i64,i64,i64,i64,i64,f64,str,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,f64,i64,i64,str,i64,bool,i64
12,44,2,"""KV Kortrijk""",806,355,5,9.75,"""2000-02-08""",7,22,249,1153,54,3,14,0,1,0,0,189,1.28,8.47,"""AM""",null,null,23.07,2,11,1006,137,980,997,"""2022-02-02""",3,19,7,54,26,1,8,6382,658,69.43,60031,74,0,2,0,8,"""J. Herrmann""",17,5,148,2,65,1006,3477,"""2021/2022""",15,92757,78,220,0,83,7.95,7,14,12,9,2795,0,189,78,24,0,8,7379,298,61.48,60,0,"""Midfield""",433,33,83,0,16,22,26.7,"""Jesaja Herrmann""",14,"""KV Kortrijk""",1,5,65,4,0,8,51,37,77,3,0,5.0,682,1,8,1,0,28.07,1,3,"""KV Kortrijk v Royal Antwerp FC""",173,true,1
15,18,null,"""KV Oostende""",371,137,2,22.32,"""1997-12-04""",8,15,249,916,0,4,24,11,1,10,2,314,16.07,6.25,"""CF""",2,1,1.92,2,4,688,144,250,5050,"""2021-07-30""",0,29,24,76,10,2,15,1565,0,61.17,12287,64,30,8,1,4,"""M. Gueye""",0,15,371,6,49,2605,1677,"""2021/2022""",10,45284,322,0,57,131,46.15,18,36,5,47,0,4,89,248,8,1917,8,6615,248,15.02,108,1,"""Center Forward""",459,34,74,6,16,0,29.0,"""Makhtar Gueye""",54,"""KV Oostende""",3,6,17,6,76,11,13,0,36,2,225,11.58,1677,3,5,1,1,13.5,2,4,"""KRC Genk v KV Oostende""",666,true,null
32,12,null,"""Royal Antwerp FC""",320,98,2,25.54,"""2003-01-30""",5,13,249,1249,0,0,69,18,1,20,0,771,18.27,7.27,"""LW""",4,0,1.95,0,4,1102,155,3

In [21]:
corr_df_missing.describe()

statistic,Count Medium Acceleration OTIP,Count Medium Acceleration TIP,Count High Deceleration TIP 2,team_name,Running Distance TIP,HSR Distance 2,Count Sprint 2,Minutes OTIP,player_birthdate,Count Sprint,Count HSR TIP,competition_edition_id,Running Distance,Sprinting Distance TIP 2,Count Sprint TIP,Sprinting Distance OTIP 2,Count Medium Deceleration OTIP 1,Count High Acceleration TIP,Count Medium Acceleration OTIP 1,Count High Acceleration OTIP,Running Distance OTIP,Minutes OTIP 1,Minutes OTIP 2,position,Count High Deceleration OTIP 1,Count High Acceleration OTIP 1,Minutes TIP 2,Count High Acceleration 2,Count High Deceleration 2,Distance OTIP 2,Sprinting Distance,Running Distance 2,Distance 1,date,Count Sprint TIP 2,team_id,Count HSR 1,Sprinting Distance TIP,Count HSR 2,Count High Deceleration TIP 1,Count Medium Deceleration OTIP,Distance 2,Running Distance TIP 2,Minutes,player_id,Count Medium Acceleration,Sprinting Distance OTIP 1,Count Medium Deceleration TIP 1,Count High Acceleration OTIP 2,Count Medium Deceleration OTIP 2,short_name,Count HSR TIP 2,Count HSR TIP 1,Running Distance TIP 1,Count Sprint 1,Count Medium Deceleration,Distance OTIP,Distance TIP,season_name,Count High Deceleration,match_id,HSR Distance 1,HSR Distance TIP 2,HSR Distance OTIP 1,HSR Distance OTIP,Minutes 1,Count Medium Acceleration TIP 1,Count Medium Deceleration 1,Count Medium Acceleration OTIP 2,Count Medium Acceleration 1,Distance TIP 2,Count Sprint TIP 1,Running Distance OTIP 2,HSR Distance TIP 1,Count Medium Deceleration TIP,Distance OTIP 1,season_id,Distance,HSR Distance TIP,Minutes 2,Sprinting Distance 1,Count High Acceleration TIP 1,group,HSR Distance,Count HSR,HSR Distance OTIP 2,Count HSR OTIP 1,competition_id,Count Medium Deceleration TIP 2,PSV-99,player_name,Sprinting Distance OTIP,team,Count High Deceleration OTIP,Count High Acceleration,Count Medium Acceleration 2,Count High Deceleration 1,Sprinting Distance TIP 1,Count HSR OTIP,Count Medium Deceleration 2,Count Medium Acceleration TIP 2,Sprinting Distance 2,Count High Deceleration TIP,Running Distance OTIP 1,Minutes TIP 1,Distance TIP 1,Count Sprint OTIP,Count HSR OTIP 2,Count High Deceleration OTIP 2,Count Sprint OTIP 1,Minutes TIP,Count Sprint OTIP 2,Count High Acceleration 1,match_name,Running Distance 1,quality_check,Count High Acceleration TIP 2
str,f64,f64,f64,str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64
"""count""",1824.0,1824.0,1034.0,"""1824""",1824.0,1798.0,1798.0,1824.0,"""1611""",1824.0,1824.0,1824.0,1824.0,1798.0,1824.0,1798.0,1716.0,1824.0,1716.0,1824.0,1824.0,1824.0,1824.0,"""1824""",1606.0,1606.0,1824.0,1798.0,1798.0,1798.0,1824.0,1798.0,1716.0,"""1824""",1798.0,1824.0,1716.0,1824.0,1798.0,1586.0,1824.0,1798.0,1798.0,1824.0,1824.0,1824.0,1716.0,1716.0,1271.0,1798.0,"""1824""",1798.0,1716.0,1716.0,1716.0,1824.0,1824.0,1824.0,"""1824""",1824.0,1824.0,1716.0,1798.0,1716.0,1824.0,1824.0,1716.0,1716.0,1798.0,1716.0,1798.0,1716.0,1798.0,1716.0,1824.0,1716.0,1824.0,1824.0,1824.0,1824.0,1716.0,1586.0,"""1824""",1824.0,1824.0,1798.0,1716.0,1824.0,1798.0,1824.0,"""1824""",1824.0,"""1824""",1824.0,1824.0,1798.0,1716.0,1716.0,1824.0,1798.0,1798.0,1798.0,1824.0,1716.0,1824.0,1716.0,1824.0,1798.0,1271.0,1716.0,1824.0,1798.0,1716.0,"""1824""",1716.0,1824.0,1034.0
"""null_count""",0.0,0.0,790.0,"""0""",0.0,26.0,26.0,0.0,"""213""",0.0,0.0,0.0,0.0,26.0,0.0,26.0,108.0,0.0,108.0,0.0,0.0,0.0,0.0,"""0""",218.0,218.0,0.0,26.0,26.0,26.0,0.0,26.0,108.0,"""0""",26.0,0.0,108.0,0.0,26.0,238.0,0.0,26.0,26.0,0.0,0.0,0.0,108.0,108.0,553.0,26.0,"""0""",26.0,108.0,108.0,108.0,0.0,0.0,0.0,"""0""",0.

In [22]:
#removing the rows containing null values
corr_df_clean = corr_df.drop_nulls()
corr_df_clean.shape

# deleted 1824 rows

(447713, 117)

In [23]:
corr_df_clean.describe()
#moeilijk de vinger op te leggen waardoor 0 waarden waren binnen de kolommen.
#lijkt wel dat er geen minute koloomen meer zijn met waarden gelijk aan 0


statistic,Count Medium Acceleration OTIP,Count Medium Acceleration TIP,Count High Deceleration TIP 2,team_name,Running Distance TIP,HSR Distance 2,Count Sprint 2,Minutes OTIP,player_birthdate,Count Sprint,Count HSR TIP,competition_edition_id,Running Distance,Sprinting Distance TIP 2,Count Sprint TIP,Sprinting Distance OTIP 2,Count Medium Deceleration OTIP 1,Count High Acceleration TIP,Count Medium Acceleration OTIP 1,Count High Acceleration OTIP,Running Distance OTIP,Minutes OTIP 1,Minutes OTIP 2,position,Count High Deceleration OTIP 1,Count High Acceleration OTIP 1,Minutes TIP 2,Count High Acceleration 2,Count High Deceleration 2,Distance OTIP 2,Sprinting Distance,Running Distance 2,Distance 1,date,Count Sprint TIP 2,team_id,Count HSR 1,Sprinting Distance TIP,Count HSR 2,Count High Deceleration TIP 1,Count Medium Deceleration OTIP,Distance 2,Running Distance TIP 2,Minutes,player_id,Count Medium Acceleration,Sprinting Distance OTIP 1,Count Medium Deceleration TIP 1,Count High Acceleration OTIP 2,Count Medium Deceleration OTIP 2,short_name,Count HSR TIP 2,Count HSR TIP 1,Running Distance TIP 1,Count Sprint 1,Count Medium Deceleration,Distance OTIP,Distance TIP,season_name,Count High Deceleration,match_id,HSR Distance 1,HSR Distance TIP 2,HSR Distance OTIP 1,HSR Distance OTIP,Minutes 1,Count Medium Acceleration TIP 1,Count Medium Deceleration 1,Count Medium Acceleration OTIP 2,Count Medium Acceleration 1,Distance TIP 2,Count Sprint TIP 1,Running Distance OTIP 2,HSR Distance TIP 1,Count Medium Deceleration TIP,Distance OTIP 1,season_id,Distance,HSR Distance TIP,Minutes 2,Sprinting Distance 1,Count High Acceleration TIP 1,group,HSR Distance,Count HSR,HSR Distance OTIP 2,Count HSR OTIP 1,competition_id,Count Medium Deceleration TIP 2,PSV-99,player_name,Sprinting Distance OTIP,team,Count High Deceleration OTIP,Count High Acceleration,Count Medium Acceleration 2,Count High Deceleration 1,Sprinting Distance TIP 1,Count HSR OTIP,Count Medium Deceleration 2,Count Medium Acceleration TIP 2,Sprinting Distance 2,Count High Deceleration TIP,Running Distance OTIP 1,Minutes TIP 1,Distance TIP 1,Count Sprint OTIP,Count HSR OTIP 2,Count High Deceleration OTIP 2,Count Sprint OTIP 1,Minutes TIP,Count Sprint OTIP 2,Count High Acceleration 1,match_name,Running Distance 1,quality_check,Count High Acceleration TIP 2
str,f64,f64,f64,str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64
"""count""",447713.0,447713.0,447713.0,"""447713""",447713.0,447713.0,447713.0,447713.0,"""447713""",447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,"""447713""",447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,"""447713""",447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,"""447713""",447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,"""447713""",447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,"""447713""",447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,"""447713""",447713.0,"""447713""",447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,"""447713""",447713.0,447713.0,447713.0
"""null_count""",0.0,0.0,0.0,"""0""",0.0,0.0,0.0,0.0,"""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [24]:
# chechk if all null values are gone
corr_df_missing = (corr_df_clean.filter(pl.any_horizontal(pl.all().is_null())))
corr_df_missing.shape

(0, 117)

#### Calculating correlations

In [25]:
#filter out the numerical columns of the dataframe to calculate correlations
num_corr_df = corr_df_clean.select(
    pl.col(pl.Int64),
    pl.col(pl.Float64)
    )
num_corr_df.describe()

statistic,Count Medium Acceleration OTIP,Count Medium Acceleration TIP,Count High Deceleration TIP 2,Running Distance TIP,HSR Distance 2,Count Sprint 2,Count Sprint,Count HSR TIP,competition_edition_id,Running Distance,Sprinting Distance TIP 2,Count Sprint TIP,Sprinting Distance OTIP 2,Count Medium Deceleration OTIP 1,Count High Acceleration TIP,Count Medium Acceleration OTIP 1,Count High Acceleration OTIP,Running Distance OTIP,Count High Deceleration OTIP 1,Count High Acceleration OTIP 1,Count High Acceleration 2,Count High Deceleration 2,Distance OTIP 2,Sprinting Distance,Running Distance 2,Distance 1,Count Sprint TIP 2,team_id,Count HSR 1,Sprinting Distance TIP,Count HSR 2,Count High Deceleration TIP 1,Count Medium Deceleration OTIP,Distance 2,Running Distance TIP 2,player_id,Count Medium Acceleration,Sprinting Distance OTIP 1,Count Medium Deceleration TIP 1,Count High Acceleration OTIP 2,Count Medium Deceleration OTIP 2,Count HSR TIP 2,Count HSR TIP 1,Running Distance TIP 1,Count Sprint 1,Count Medium Deceleration,Distance OTIP,Distance TIP,Count High Deceleration,match_id,HSR Distance 1,HSR Distance TIP 2,HSR Distance OTIP 1,HSR Distance OTIP,Count Medium Acceleration TIP 1,Count Medium Deceleration 1,Count Medium Acceleration OTIP 2,Count Medium Acceleration 1,Distance TIP 2,Count Sprint TIP 1,Running Distance OTIP 2,HSR Distance TIP 1,Count Medium Deceleration TIP,Distance OTIP 1,season_id,Distance,HSR Distance TIP,Sprinting Distance 1,Count High Acceleration TIP 1,HSR Distance,Count HSR,HSR Distance OTIP 2,Count HSR OTIP 1,competition_id,Count Medium Deceleration TIP 2,Sprinting Distance OTIP,Count High Deceleration OTIP,Count High Acceleration,Count Medium Acceleration 2,Count High Deceleration 1,Sprinting Distance TIP 1,Count HSR OTIP,Count Medium Deceleration 2,Count Medium Acceleration TIP 2,Sprinting Distance 2,Count High Deceleration TIP,Running Distance OTIP 1,Distance TIP 1,Count Sprint OTIP,Count HSR OTIP 2,Count High Deceleration OTIP 2,Count Sprint OTIP 1,Count Sprint OTIP 2,Count High Acceleration 1,Running Distance 1,Count High Acceleration TIP 2,Minutes OTIP,Minutes OTIP 1,Minutes OTIP 2,Minutes TIP 2,Minutes,Minutes 1,Minutes 2,PSV-99,Minutes TIP 1,Minutes TIP
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [26]:
#filtering on outliers
num_corr_df_np = num_corr_df.to_numpy()
zscores = zscore(num_corr_df_np)

In [27]:
print(zscores)

[[ 1.19416358 -0.60705378 -0.18409922 ... -1.48827036 -0.48332657
  -0.3957259 ]
 [-0.10919564 -0.29512735 -0.18409922 ... -0.07587984 -0.46165608
  -0.18964274]
 [-0.89121118 -0.19115187 -0.18409922 ... -1.10801137 -0.05301264
  -0.67260099]
 ...
 [ 1.62861665 -0.71102926  0.81128293 ... -0.56478425  0.52280312
  -0.1094119 ]
 [-0.28297687  0.12077457 -1.17948138 ...  0.52166999  0.27514043
  -0.96678076]
 [ 0.58592927 -0.08717639  0.81128293 ... -0.02155713 -0.49261392
  -0.66001575]]


In [40]:
# calculating Z-scores
zscore_array = (np.abs(zscore(num_corr_df.to_numpy())) >= 3).all(axis=1)

In [42]:
# checking if there are any extreme outliers
# there are no outliers that are more than 3 standard deviations away from the variable mean 
outlier_number = 0

for boolean in tqdm(zscore_array):
    if boolean == True:
        outlier_number += 1
    else:
        continue

print(outlier_number)

100%|██████████| 447713/447713 [00:02<00:00, 194452.11it/s]

0


In [76]:
#initialize empty dataframe to store correlations
correlations_df = pl.DataFrame(schema=[
    ("var1", pl.Utf8),
    ("var2", pl.Utf8),
    ("corr", pl.Float64)
])

In [81]:
correlations = num_corr_df.corr()

columns = correlations.columns

for i in range(len(columns)):
    for j in range(i + 1, len(columns)):  # Only upper triangle to avoid repetition
        correlation = correlations[i, j]

        if correlation >0.8 or correlation < -0.8:
            correlations_df.extend(pl.DataFrame({"var1":[columns[i]], "var2":[columns[j]], "corr":[correlations[i, j]]}))
            #print(f"Correlation between {columns[i]} and {columns[j]}: {correlations[i, j]}")
        else:
            continue

In [82]:
correlations.shape

(106, 106)

In [83]:
correlations.head()

Count Medium Deceleration TIP 2,Count High Deceleration,Distance 1,Distance TIP 2,Count High Acceleration OTIP 2,competition_id,Count High Deceleration 1,Count HSR OTIP 2,Count Medium Deceleration 1,match_id,Count Sprint TIP 1,Running Distance,Count High Deceleration OTIP,Distance 2,Running Distance 1,Count High Acceleration OTIP 1,Sprinting Distance OTIP 2,Count Sprint OTIP,Running Distance OTIP,Sprinting Distance TIP,Count Medium Acceleration 1,Running Distance TIP,Sprinting Distance 1,Sprinting Distance OTIP 1,Count Medium Acceleration,Count Sprint 1,player_id,Count Medium Deceleration 2,Count High Deceleration OTIP 2,Count HSR TIP,Count Medium Deceleration,HSR Distance OTIP 2,Sprinting Distance,Count Medium Deceleration OTIP 1,Count High Deceleration OTIP 1,Distance OTIP 2,Count Medium Deceleration TIP,Count Sprint TIP,Count Medium Acceleration OTIP 2,Count High Acceleration TIP 1,Count HSR TIP 1,HSR Distance 1,Count HSR,Count Medium Acceleration OTIP 1,competition_edition_id,Count HSR TIP 2,Count Sprint OTIP 1,Running Distance TIP 1,Count High Acceleration,Count Sprint TIP 2,Running Distance OTIP 2,season_id,Distance TIP 1,Count Medium Acceleration OTIP,HSR Distance TIP 1,HSR Distance TIP 2,Count Medium Acceleration TIP 2,Count High Deceleration TIP,HSR Distance OTIP 1,Count Medium Acceleration TIP 1,Count HSR OTIP 1,Count High Acceleration TIP,Running Distance 2,Count Medium Acceleration 2,Count High Acceleration TIP 2,Count Medium Acceleration TIP,Count HSR 2,Sprinting Distance TIP 1,Count HSR 1,Count Sprint,Running Distance OTIP 1,HSR Distance TIP,Count High Acceleration 2,HSR Distance OTIP,Count High Acceleration OTIP,Count High Deceleration TIP 2,Distance OTIP,Sprinting Distance 2,HSR Distance,Count Sprint OTIP 2,Count Medium Deceleration OTIP,Count Medium Deceleration OTIP 2,team_id,Distance,Count Medium Deceleration TIP 1,Count Sprint 2,Count High Deceleration TIP 1,Sprinting Distance TIP 2,Count HSR OTIP,Running Distance TIP 2,Count High Acceleration 1,HSR Distance 2,Sprinting Distance OTIP,Distance OTIP 1,Distance TIP,Count High Deceleration 2,Minutes TIP,Minutes TIP 2,Minutes 2,Minutes OTIP,PSV-99,Minutes 1,Minutes OTIP 2,Minutes TIP 1,Minutes,Minutes OTIP 1
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1.0,0.163882,0.023148,0.879156,0.074861,-0.020147,-0.035865,0.263566,0.104883,-0.027993,-0.02356,0.338757,-0.009325,0.604831,0.058148,-0.046837,0.155873,0.089844,0.096941,0.068818,0.108258,0.492987,-0.031283,-0.011873,0.406467,-0.023726,-0.040909,0.61478,0.102207,0.246042,0.481798,0.247416,0.103109,-0.127552,-0.09493,0.154857,0.842022,0.10154,0.146324,0.04999,0.028484,0.005044,0.264108,-0.041508,-0.023142,0.452486,-0.012929,0.159696,0.16344,0.228371,0.212976,-0.007109,0.295529,0.065446,0.026132,0.409489,0.836612,0.33584,-0.025573,0.254654,-0.016557,0.230566,0.511301,0.535621,0.337088,0.651316,0.456559,-0.036345,0.009302,0.129526,-0.045131,0.226633,0.274671,0.134311,0.016884,0.476144,0.003936,0.221357,0.250138,0.162016,-0.016905,0.089066,-0.057041,0.553691,0.337203,0.26378,0.086357,0.175583,0.152746,0.696108,0.012411,0.428736,0.08953,-0.191347,0.772167,0.31786,0.763062,0.847504,0.554932,-0.033486,-0.022095,-0.054286,0.123365,0.305618,0.529475,-0.236716
0.163882,1.0,0.182597,0.151515,0.287176,-0.063003,0.831845,0.282013,0.238522,-0.002845,0.439909,0.290163,0.732962,0.178281,0.210964,0.282706,0.22368,0.357596,0.174284,0.491913,0.306399,0.307534,0.435974,0.238277,0.360236,0.485371,-0.004693,0.273972,0.574024,0.529083,0.318869,0.269209,0.518106,0.12327,0.600287,0.135735,0.143767,0.525344,0.106238,0.358807,0.442904,0.455421,0.596927,0.102775,-0.015

In [84]:
pl.Config.set_tbl_rows(150)
correlations_df

var1,var2,corr
str,str,f64
"""Count Medium Deceleration TIP …","""Distance TIP 2""",0.879156
"""Count Medium Deceleration TIP …","""Count Medium Deceleration TIP""",0.842022
"""Count Medium Deceleration TIP …","""Count Medium Acceleration TIP …",0.836612
"""Count Medium Deceleration TIP …","""Minutes TIP 2""",0.847504
"""Count High Deceleration""","""Count High Deceleration 1""",0.831845
"""Distance 1""","""Running Distance 1""",0.837679
"""Distance TIP 2""","""Distance TIP""",0.856596
"""Distance TIP 2""","""Minutes TIP""",0.823531
"""Distance TIP 2""","""Minutes TIP 2""",0.942187


In [ ]:
# pygwalk can be used as easy tool for quick data inspection 
pyg.walk(position_test_df)

## Building the final data frame
For further analysis I wanted to incorporate the data from the ce dataframe into the cleaned combined_json_df

In [83]:
combined_json_df = corr_df_clean
#combined_json_df.shape

ce_df = ce
#ce_df.shape

In [49]:
combined_json_df.head()

Count Medium Acceleration OTIP,Count Medium Acceleration TIP,Count High Deceleration TIP 2,team_name,Running Distance TIP,HSR Distance 2,Count Sprint 2,Minutes OTIP,player_birthdate,Count Sprint,Count HSR TIP,competition_edition_id,Running Distance,Sprinting Distance TIP 2,Count Sprint TIP,Sprinting Distance OTIP 2,Count Medium Deceleration OTIP 1,Count High Acceleration TIP,Count Medium Acceleration OTIP 1,Count High Acceleration OTIP,Running Distance OTIP,Minutes OTIP 1,Minutes OTIP 2,position,Count High Deceleration OTIP 1,Count High Acceleration OTIP 1,Minutes TIP 2,Count High Acceleration 2,Count High Deceleration 2,Distance OTIP 2,Sprinting Distance,Running Distance 2,Distance 1,date,Count Sprint TIP 2,team_id,Count HSR 1,Sprinting Distance TIP,Count HSR 2,Count High Deceleration TIP 1,Count Medium Deceleration OTIP,Distance 2,Running Distance TIP 2,Minutes,player_id,Count Medium Acceleration,Sprinting Distance OTIP 1,Count Medium Deceleration TIP 1,Count High Acceleration OTIP 2,Count Medium Deceleration OTIP 2,short_name,Count HSR TIP 2,Count HSR TIP 1,Running Distance TIP 1,Count Sprint 1,Count Medium Deceleration,Distance OTIP,Distance TIP,season_name,Count High Deceleration,match_id,HSR Distance 1,HSR Distance TIP 2,HSR Distance OTIP 1,HSR Distance OTIP,Minutes 1,Count Medium Acceleration TIP 1,Count Medium Deceleration 1,Count Medium Acceleration OTIP 2,Count Medium Acceleration 1,Distance TIP 2,Count Sprint TIP 1,Running Distance OTIP 2,HSR Distance TIP 1,Count Medium Deceleration TIP,Distance OTIP 1,season_id,Distance,HSR Distance TIP,Minutes 2,Sprinting Distance 1,Count High Acceleration TIP 1,group,HSR Distance,Count HSR,HSR Distance OTIP 2,Count HSR OTIP 1,competition_id,Count Medium Deceleration TIP 2,PSV-99,player_name,Sprinting Distance OTIP,team,Count High Deceleration OTIP,Count High Acceleration,Count Medium Acceleration 2,Count High Deceleration 1,Sprinting Distance TIP 1,Count HSR OTIP,Count Medium Deceleration 2,Count Medium Acceleration TIP 2,Sprinting Distance 2,Count High Deceleration TIP,Running Distance OTIP 1,Minutes TIP 1,Distance TIP 1,Count Sprint OTIP,Count HSR OTIP 2,Count High Deceleration OTIP 2,Count Sprint OTIP 1,Minutes TIP,Count Sprint OTIP 2,Count High Acceleration 1,match_name,Running Distance 1,quality_check,Count High Acceleration TIP 2
i64,i64,i64,str,i64,i64,i64,f64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,str,i64,i64,f64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,str,i64,i64,i64,i64,i64,i64,f64,str,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,f64,i64,i64,str,i64,bool,i64
47,22,1,"""Royal Antwerp FC""",661,358,2,35.57,"""1995-02-17""",5,19,249,2415,19,2,18,17,0,24,0,1504,18.27,17.3,"""AM""",3,0,11.08,1,3,2744,148,1061,6810,"""2021-07-25""",1,462,33,30,26,2,34,6216,292,96.63,8701,100,27,8,0,17,"""P. Gerkens""",6,13,369,3,89,5727,3432,"""2021/2022""",11,44999,387,121,225,443,47.15,12,42,23,52,1558,1,662,139,17,2983,8,13026,260,49.48,92,0,"""Midfield""",745,59,218,17,16,9,25.7,"""Pieter Gerkens""",45,"""Royal Antwerp FC""",4,4,48,8,11,34,47,10,56,3,842,13.13,1874,2,17,1,1,24.21,1,3,"""KV Mechelen v Royal Antwerp FC""",1354,true,0
32,25,1,"""Oud-Heverlee Leuven""",610,275,5,29.59,"""1989-07-25""",8,20,249,1101,119,4,0,11,2,15,1,402,14.52,15.07,"""AM""",1,0,12.32,2,3,1739,202,612,4842,"""2021-07-24""",3,27,20,136,19,1,25,5127,343,96.19,10932,77,24,9,1,14,"""X. Mercier""",11,9,267,3,77,3534,3435,"""2021/2022""",8,45002,201,184,104,167,46.12,11,31,17,36,1775,1,224,81,20,1795,8,9969,265,50.07,48,1,"""Midfield""",476,39,63,10,16,11,28.3,"""Xavier Mercier""",24,"""Oud-Heverlee Leuven""",1,5,41,5,17,17,46,14,154,2,178,13.2,1660,1,7,0,1,25.52,0,3,"""Oud-Heverlee Leuven v SV Zulte…",489,true,1
23,26,1,"""SV Zulte-Waregem""",724,129,1,19.77,"""1996-01-18""",5,18,249,124

In [87]:
#combined_json_df.select("competition_id").unique()

competition_id
i64
140
90
194
3
18
42
93
87
57


In [90]:
#dropping ce_df columns that are already present in combined_json_df so there is no overlap
ce_df = ce_df.drop(["season_id","season_name", "competition_id"])

In [92]:
# renaming ce_df columns so they don't cause confusion when merging with the combined_json_df
ce_df = ce_df.rename({
    "name":"full_comp_name",
    "id":"ec_id",
})

In [93]:
ce_df.head()

full_comp_name,ec_id,season_start_year,season_end_year,competition_area,competition_name,competition_gender,competition_age_group
str,i64,i64,i64,str,str,str,str
"""FIFA - World Cup - 2014""",0,2014,2014,"""FIFA""","""World Cup""","""male""","""adult"""
"""ENG - Premier League - 2015/20…",1,2015,2016,"""ENG""","""Premier League""","""male""","""adult"""
"""UEFA - European Championship -…",2,2016,2016,"""UEFA""","""European Championship""","""male""","""adult"""
"""ENG - Premier League - 2016/20…",3,2016,2017,"""ENG""","""Premier League""","""male""","""adult"""
"""ENG - Premier League - 2017/20…",4,2017,2018,"""ENG""","""Premier League""","""male""","""adult"""


In [88]:
#combined_json_df.select("competition_id").unique()

competition_id
i64
18
140
57
12
194
87
30
3
90


In [63]:
bundeled_df = combined_json_df.with_columns(
    pl.lit(None).alias("full_comp_name"),
    pl.lit(None).alias("ec_id"),
    pl.lit(None).alias("season_start_year"),
    pl.lit(None).alias("season_end_year"),
    pl.lit(None).alias("competition_area"),
    pl.lit(None).alias("competition_name"),
    pl.lit(None).alias("competition_gender"),
    pl.lit(None).alias("competition_age_group"),
)

In [65]:
#bundeled_df

In [95]:
# combined_json_df (competition_edition_id) and ce_df (id) match 
# intialize a list to save all dictionaries + and then concat them into a dataframe (as before)
found = False

for json_row in combined_json_df.iter_rows(named=True):
    for ce_row in ce_df.iter_rows(named=True):
        if json_row["competition_edition_id"] == ce_row["ec_id"]:
            json_row.update(ce_row)
            print(json_row)
            #print(ce_row)
            print("succes")
            found = True
            break
    if found:
        break

{'Count Medium Acceleration OTIP': 47, 'Count Medium Acceleration TIP': 22, 'Count High Deceleration TIP 2': 1, 'team_name': 'Royal Antwerp FC', 'Running Distance TIP': 661, 'HSR Distance 2': 358, 'Count Sprint 2': 2, 'Minutes OTIP': 35.57, 'player_birthdate': '1995-02-17', 'Count Sprint': 5, 'Count HSR TIP': 19, 'competition_edition_id': 249, 'Running Distance': 2415, 'Sprinting Distance TIP 2': 19, 'Count Sprint TIP': 2, 'Sprinting Distance OTIP 2': 18, 'Count Medium Deceleration OTIP 1': 17, 'Count High Acceleration TIP': 0, 'Count Medium Acceleration OTIP 1': 24, 'Count High Acceleration OTIP': 0, 'Running Distance OTIP': 1504, 'Minutes OTIP 1': 18.27, 'Minutes OTIP 2': 17.3, 'position': 'AM', 'Count High Deceleration OTIP 1': 3, 'Count High Acceleration OTIP 1': 0, 'Minutes TIP 2': 11.08, 'Count High Acceleration 2': 1, 'Count High Deceleration 2': 3, 'Distance OTIP 2': 2744, 'Sprinting Distance': 148, 'Running Distance 2': 1061, 'Distance 1': 6810, 'date': '2021-07-25', 'Count Sp